In [1]:
import pandas as pd
import spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
train_data = pd.read_csv("/content/drive/MyDrive/dataset/word2vec/dataset/treino.csv")
train_data.sample(5)

,title,text,date,category,subcategory,link
7591,Mega-Sena acumula e deve pagar prêmio de R$ 39...,"Sem vencedores, a Mega-Sena acumulou e deve pa...",2015-10-31,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/10...
41154,"'Eles só vão parar se forem mortos', diz em SP...","Sobrevivente do atentado ao ""Charlie Hebdo"", L...",2015-03-07,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
1998,Nova peça de Jô Soares é farsa surreal que que...,"""Histeria"" é uma farsa na vertente inglesa rec...",2016-05-27,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2016/05...
17827,Cunha devolve pedido de impeachment para Bicud...,"O presidente da Câmara dos Deputados, Eduardo ...",2015-09-14,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
84392,Ministério do Esporte exonera executivo que tr...,O secretário-executivo do Ministério do Esport...,2015-11-17,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/11/1...


In [36]:
#execute just once
# !python -m spacy download pt_core_news_sm
!pip install gensim==4.1.2
!pip install spacy==3.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 28.5 MB/s eta 0:00:00
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.1/671.1 KB 34.7 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.7.0
    Uninstalling typer-0.7.0:
      Successfully uninstalled typer-0.7.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.2
    Uninstalling pydantic-1.10.2:
      Successfully uninstalled pydantic-1.10.2
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.6
    Uninstalling thinc-8.1.6:
      Successfully uninstalled thinc-8.1.6
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently t

In [4]:
nlp = spacy.load("pt_core_news_sm")

In [5]:
text = "Rio de Janeiro é uma cidade maravilhosa"
doc = nlp(text)
doc

Rio de Janeiro é uma cidade maravilhosa

In [6]:
type(doc[2])

spacy.tokens.token.Token

In [7]:
texts_to_treatment = (titles.lower() for titles in train_data["title"])

In [8]:
def treat_texts(doc):
    valid_tokens = []
    for token in doc:
        is_valid = not token.is_stop and token.is_alpha
        if is_valid:
            valid_tokens.append(token.text)

    if len(valid_tokens) > 2:
        return  " ".join(valid_tokens)

text = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(text)
treat_texts(doc)

'Rio Janeiro cidade maravilhosa'

In [9]:
text = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(text)
treat_texts(doc)

'Rio Janeiro cidade maravilhosa'

In [10]:
from time import time

t0 = time()
treated_texts = [treat_texts(doc) for doc in nlp.pipe(texts_to_treatment,
                                                        batch_size = 1000,
                                                        n_process = -1)]
tf = time() - t0
print(tf/60)

3.5555718620618184


In [11]:
treated_titles = pd.DataFrame({"titulo": treated_texts})
treated_titles.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [12]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [13]:
print(len(treated_titles))
treated_titles = treated_titles.dropna().drop_duplicates()
print(len(treated_titles))

90000
84466


In [14]:
list_tokens_list = [title.split(" ") for title in treated_titles.titulo]

In [15]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_model = Word2Vec(sg = 0,
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_model.build_vocab(list_tokens_list, progress_per=5000)

In [16]:
dir(w2v_model)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_input_data_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_job_params',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_minimize_model',
 '_raw_word_count',
 '_save_specials',
 '_set_train_params',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_update_job_params',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'accuracy',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'ca

In [17]:
w2v_model.corpus_count

84466

In [18]:
w2v_model.train(list_tokens_list, 
                 total_examples=w2v_model.corpus_count,
                 epochs = 30)

(14584049, 16207260)

In [19]:
w2v_model.wv.most_similar("google")

[('apple', 0.6033927202224731),
 ('facebook', 0.5326541662216187),
 ('amazon', 0.4996454119682312),
 ('uber', 0.48260077834129333),
 ('volkswagen', 0.48259082436561584),
 ('fbi', 0.4671590328216553),
 ('yahoo', 0.45725882053375244),
 ('airbnb', 0.45679813623428345),
 ('software', 0.4548301100730896),
 ('waze', 0.4520329535007477)]

In [20]:
w2v_model.wv.most_similar("microsoft")

[('amazon', 0.6000848412513733),
 ('unilever', 0.5877584218978882),
 ('sony', 0.5572519302368164),
 ('tesla', 0.5451008081436157),
 ('viajante', 0.5314162969589233),
 ('canais', 0.5303372740745544),
 ('buffett', 0.5236181020736694),
 ('braskem', 0.5209256410598755),
 ('walmart', 0.5207629203796387),
 ('editora', 0.5138778686523438)]

In [21]:
w2v_model.wv.most_similar("barcelona")

[('barça', 0.5767088532447815),
 ('bayern', 0.5634491443634033),
 ('chelsea', 0.5521875619888306),
 ('madrid', 0.5416667461395264),
 ('psg', 0.5367766618728638),
 ('juventus', 0.535883903503418),
 ('lazio', 0.5228630304336548),
 ('munique', 0.5227996110916138),
 ('betis', 0.5210447311401367),
 ('leicester', 0.5149264931678772)]

In [22]:
w2v_model.wv.most_similar("messi")

[('suárez', 0.6455313563346863),
 ('cristiano', 0.5675351619720459),
 ('cavani', 0.5618014931678772),
 ('tevez', 0.5187292695045471),
 ('chuteiras', 0.5067616701126099),
 ('barça', 0.5001705884933472),
 ('ronaldo', 0.4978989362716675),
 ('benzema', 0.494916170835495),
 ('enrique', 0.48295801877975464),
 ('neymar', 0.4825179874897003)]

In [23]:
w2v_model.wv.most_similar("gm")

[('chrysler', 0.66728675365448),
 ('embraer', 0.6644535064697266),
 ('volks', 0.6525987386703491),
 ('ab', 0.6148461103439331),
 ('honda', 0.6146793365478516),
 ('braskem', 0.6059238910675049),
 ('fiat', 0.599651038646698),
 ('inbev', 0.5954645872116089),
 ('csn', 0.5847465991973877),
 ('volkswagen', 0.5763612985610962)]

In [24]:
#model Skip-Gram training
w2v_model_sg = Word2Vec(sg = 1,
                      window = 5,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_model_sg.build_vocab(list_tokens_list, progress_per=5000)

w2v_model_sg.train(list_tokens_list, 
                 total_examples=w2v_model_sg.corpus_count,
                 epochs = 30)

(14584214, 16207260)

In [25]:
w2v_model_sg.wv.most_similar("google")

[('reguladores', 0.4361687898635864),
 ('android', 0.411972314119339),
 ('apple', 0.40049201250076294),
 ('waze', 0.38362807035446167),
 ('buffett', 0.3803238570690155),
 ('facebook', 0.37379688024520874),
 ('anunciantes', 0.3645574748516083),
 ('concorda', 0.35382840037345886),
 ('ericsson', 0.3523550033569336),
 ('warren', 0.35187268257141113)]

In [26]:
w2v_model.wv.most_similar("google")

[('apple', 0.6033927202224731),
 ('facebook', 0.5326541662216187),
 ('amazon', 0.4996454119682312),
 ('uber', 0.48260077834129333),
 ('volkswagen', 0.48259082436561584),
 ('fbi', 0.4671590328216553),
 ('yahoo', 0.45725882053375244),
 ('airbnb', 0.45679813623428345),
 ('software', 0.4548301100730896),
 ('waze', 0.4520329535007477)]

In [27]:
w2v_model_sg.wv.most_similar("gm")

[('metalúrgicos', 0.5627708435058594),
 ('honda', 0.5242899656295776),
 ('motors', 0.5242859125137329),
 ('airbag', 0.4933903217315674),
 ('audi', 0.4879566729068756),
 ('cubatão', 0.4778413772583008),
 ('toyota', 0.4762125611305237),
 ('compartilhamento', 0.4742444157600403),
 ('chrysler', 0.47214317321777344),
 ('airbags', 0.4716390371322632)]

In [28]:
w2v_model.wv.most_similar("gm")

[('chrysler', 0.66728675365448),
 ('embraer', 0.6644535064697266),
 ('volks', 0.6525987386703491),
 ('ab', 0.6148461103439331),
 ('honda', 0.6146793365478516),
 ('braskem', 0.6059238910675049),
 ('fiat', 0.599651038646698),
 ('inbev', 0.5954645872116089),
 ('csn', 0.5847465991973877),
 ('volkswagen', 0.5763612985610962)]

In [29]:
w2v_model.wv.save_word2vec_format("/content/drive/MyDrive/dataset/word2vec/dataset/modelo_cbow.txt", binary=False)
w2v_model_sg.wv.save_word2vec_format("/content/drive/MyDrive/dataset/word2vec/dataset/modelo_skipgram.txt", binary=False)
train_article = pd.read_csv("/content/drive/MyDrive/dataset/word2vec/dataset/treino.csv")
test_article = pd.read_csv("/content/drive/MyDrive/dataset/word2vec/dataset/teste.csv")

In [30]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])

def tokenizer(text):
    
    doc = nlp(text)
    valid_tokens = []
    for token in doc:
        is_valid = not token.is_stop and token.is_alpha
        if is_valid:
            valid_tokens.append(token.text.lower())

    
    return valid_tokens

text = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
tokens = tokenizer(text)
print(tokens)

['rio', 'janeiro', 'cidade', 'maravilhosa']


In [31]:
import numpy as np

def combination_of_vectors_by_sum(words, model):

    result_vector = np.zeros((1,300))

    for wd in words:
        try:
            result_vector += model.get_vector(wd)

        except KeyError:
            pass
                

    return result_vector

text_vector = combination_of_vectors_by_sum(tokens, w2v_model.wv)
print(text_vector.shape)
print(text_vector)

(1, 300)
[[ 8.15644339e-01  1.53201757e+00 -1.34958699e+00 -1.45894390e+00
  -2.38423884e+00 -2.63253778e-01 -1.28783375e+00  7.47264236e-01
  -9.89749290e-01  1.45634094e+00 -2.55390704e-01  3.09719668e+00
   7.82253087e-01 -1.12246765e+00  4.52415392e-01  2.51671225e-01
   3.23454896e-01  1.71082997e+00  6.93483926e-01  4.86502849e-01
  -1.93358514e+00  6.10149413e-01  7.05410063e-01 -3.14934924e-01
  -1.25045599e+00 -5.00907585e-01  1.71393208e+00 -1.76490942e+00
   5.33213623e-01  1.34279095e+00 -4.93692666e-01  6.88969493e-01
   7.54343726e-01  1.92458704e+00  1.04163766e-01 -5.60948245e-01
  -2.15606727e-01 -5.30744791e-02 -1.05088389e+00 -3.96766797e-01
   3.27522933e-01  1.43321250e+00  2.36485666e+00 -8.99529010e-01
   5.99203840e-01 -6.69930279e-01  2.70638108e-01  9.28325593e-01
   1.18012071e+00  5.06086677e-01  9.75523993e-01 -2.69310308e+00
   1.24242492e-01  5.58465272e-01 -2.64527798e+00  1.87150693e+00
  -1.11551923e-01 -5.07955804e-01 -9.83447805e-01 -2.78500021e-02
 

In [32]:
def matrix_vectors(texts, model):
    x = len(texts)
    y = 300
    matrix = np.zeros((x,y))

    for i in range(x):
        words = tokenizer(texts.iloc[i])
        matrix[i] = combination_of_vectors_by_sum(words, model)

    return matrix

matrix_vectors_train_cbow = matrix_vectors(train_article.title, w2v_model.wv)
matrix_vectors_test_cbow = matrix_vectors(test_article.title, w2v_model.wv)
print(matrix_vectors_train_cbow.shape)
print(matrix_vectors_test_cbow.shape) 

(90000, 300)
(20513, 300)


In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classifier(model, X_train, y_train, X_test, y_test):

    RL = LogisticRegression(max_iter = 800)
    RL.fit(X_train, y_train)
    categories = RL.predict(X_test)
    results = classification_report(y_test, categories)
    print(results)
    
    return RL

RL_cbow = classifier(w2v_model.wv,
                        matrix_vectors_train_cbow,
                        train_article.category,
                        matrix_vectors_test_cbow,
                        test_article.category)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.75      6103
   cotidiano       0.63      0.80      0.70      1698
     esporte       0.92      0.87      0.89      4663
   ilustrada       0.13      0.83      0.22       131
     mercado       0.83      0.78      0.81      5867
       mundo       0.74      0.83      0.78      2051

    accuracy                           0.78     20513
   macro avg       0.68      0.80      0.69     20513
weighted avg       0.81      0.78      0.79     20513



In [38]:
matrix_vectors_train_sg = matrix_vectors(train_article.title, w2v_model_sg.wv)
matrix_vectors_test_sg = matrix_vectors(test_article.title, w2v_model_sg.wv)

RL_sg = classifier(w2v_model_sg,
                        matrix_vectors_train_sg,
                        train_article.category,
                        matrix_vectors_test_sg,
                        test_article.category)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.63      0.80      0.71      1698
     esporte       0.94      0.87      0.90      4663
   ilustrada       0.14      0.87      0.24       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.75      0.84      0.80      2051

    accuracy                           0.79     20513
   macro avg       0.69      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [39]:
import pickle

with open("/content/drive/MyDrive/dataset/word2vec/dataset/rl_cbow.pkl", "wb") as f:
    pickle.dump(RL_cbow, f)

In [40]:
with open("/content/drive/MyDrive/dataset/word2vec/dataset/rl_sg.pkl", "wb") as f:
    pickle.dump(RL_sg, f)